In [16]:
from i10_bluesky.devices.rasor.i10 import pimte
from i10_bluesky.devices.rasor.detectorSlits import *
from i10_bluesky.devices.pimteAD import HDFStatsPimte
from i10_bluesky.epics.pimte_controller import PimteController
from ophyd_async.core import DeviceCollector 
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import RunEngine
import asyncio
from dodal.beamlines.beamline_utils import device_instantiation, get_directory_provider, set_directory_provider
from ophyd_async.core._providers import DirectoryInfo
loop = asyncio.get_event_loop()
bec = BestEffortCallback()
RE = RunEngine({})
dataDir = DirectoryInfo("/scratch/i10-bluesky/", "rasor/")

In [14]:
pCon = PimteController(

TypeError: HDFStatsPimte.__init__() got multiple values for argument 'directory_provider'

In [3]:
result = loop.create_task(ds.dsd.read())

In [2]:
with DeviceCollector():
    ds = DetectorSlits('ME01D-MO-APTR',name = "detSlits")

NotConnected:  Could not connect devices. Is the bluesky event loop running? See https://blueskyproject.io/ophyd-async/main/user/explanations/event-loop-choice.html for more info.


In [4]:
result

<Task finished name='Task-33' coro=<StandardReadable.read() done, defined at /venv/lib/python3.11/site-packages/ophyd_async/core/standard_readable.py:68> result={'detSlits-dsd': {'alarm_severity': 0, 'timestamp': 1711449273.729834, 'value': 25.6000703125}}>